In [1]:
import pandas as pd
import copy
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNetCV
from sklearn.metrics import f1_score
from sklearn.preprocessing import RobustScaler, SplineTransformer
import numpy as np

In [2]:
snapshot_date = pd.to_datetime('2018-12-31')

data_path = "../data/Dataset/Motor vehicle insurance data.csv"
raw_data = pd.read_csv(data_path, delimiter=';')
raw_data.shape

(105555, 30)

In [3]:
raw_data.head()

,ID,Date_start_contract,Date_last_renewal,Date_next_renewal,Date_birth,Date_driving_licence,Distribution_channel,Seniority,Policies_in_force,Max_policies,...,Area,Second_driver,Year_matriculation,Power,Cylinder_capacity,Value_vehicle,N_doors,Type_fuel,Length,Weight
0,1,05/11/2015,05/11/2015,05/11/2016,15/04/1956,20/03/1976,0,4,1,2,...,0,0,2004,80,599,7068.0,0,P,NaN,190
1,1,05/11/2015,05/11/2016,05/11/2017,15/04/1956,20/03/1976,0,4,1,2,...,0,0,2004,80,599,7068.0,0,P,NaN,190
2,1,05/11/2015,05/11/2017,05/11/2018,15/04/1956,20/03/1976,0,4,2,2,...,0,0,2004,80,599,7068.0,0,P,NaN,190
3,1,05/11/2015,05/11/2018,05/11/2019,15/04/1956,20/03/1976,0,4,2,2,...,0,0,2004,80,599,7068.0,0,P,NaN,190
4,2,26/09/2017,26/09/2017,26/09/2018,15/04/1956,20/03/1976,0,4,2,2,...,0,0,2004,80,599,7068.0,0,P,NaN,190


# Analyze the data

## Drop the ID column

In [4]:
raw_data.drop('ID', axis=1, inplace=True)

## Check Null values

In [5]:
raw_data.isnull().sum()

Date_start_contract         0
Date_last_renewal           0
Date_next_renewal           0
Date_birth                  0
Date_driving_licence        0
Distribution_channel        0
Seniority                   0
Policies_in_force           0
Max_policies                0
Max_products                0
Lapse                       0
Date_lapse              70408
Payment                     0
Premium                     0
Cost_claims_year            0
N_claims_year               0
N_claims_history            0
R_Claims_history            0
Type_risk                   0
Area                        0
Second_driver               0
Year_matriculation          0
Power                       0
Cylinder_capacity           0
Value_vehicle               0
N_doors                     0
Type_fuel                1764
Length                  10329
Weight                      0
dtype: int64

1. Date\_lapse (70,408 missing)

* What it means: No lapse has occurred. It’s not missing—it’s informative.
* Action: Replace missing with a flag:
  * Create new binary feature: has\_lapsed = 0 if missing, 1 if not.
  * May drop the actual Date\_lapse column.

2. Type\_fuel (1,764 missing)

* What it means: Fuel type was not registered or missing from source.
* Action: insert a data from condition from other fatures or create classification model.
  * Action 1: Fill with mode ('D' or 'P')
  * Action 2: Fill the empty value from prediction model.

3. Length (10,329 missing)

* What it means: Vehicle length not available. This is a mechanical feature.
* Action:
  * If other vehicle dimensions (e.g. Power, Cylinder\_capacity, Weight) are present, can impute using a regression model.
  * If useless to predict, drop the column entirely.

✅ What to do next

* Engineer binary features like has\_lapsed.
* For Length, do an experiment: run two versions of the model (with/without it) and check if it helps.
  * If it does, keep it and consider use a regression model to impute missing values.
  * If not, drop it.

# Transform the data

In [6]:
transform_data = copy.deepcopy(raw_data)

🟨 Columns that Require Conversion

1. Type\_fuel → Categorical: 'P', 'D'
   
   Convert to integer:
   * 'P' → 0
   * 'D' → 1
   * fill the null with prediction

2. Date columns (must be transformed):

   * Date\_start\_contract
   * Date\_last\_renewal
   * Date\_birth
   * Date\_driving\_licence

These need to be converted to datetime objects and engineered into features:

ex. New features:
* licence\_years = Date\_start\_contract − Date\_driving\_licence
* lapse\_flag = 1 if Date\_lapse not null else 0

Then can drop raw date columns.

## Date related features

In [7]:
# Convert date strings to datetime objects
date_cols = ["Date_start_contract", "Date_last_renewal", "Date_birth", "Date_driving_licence"]
for col in date_cols:
    transform_data[col] = pd.to_datetime(transform_data[col], format='%d/%m/%Y', errors='coerce')
    null_count = transform_data[col].isnull().sum()
    if null_count > 0:
        print(f"Column {col} has {null_count} null values after conversion to datetime.")

if not transform_data['Date_last_renewal'].max() <= snapshot_date:
    raise ValueError("Data contains post-2018 dates - inconsistent with dataset description")

## categorical
Type\_fuel → Categorical: 'P', 'D' to 0, 1

left the empty values as it is, for now

In [8]:
# Create a mapping dictionary
fuel_type_map = {'P': 0, 'D': 1}

# Replace values using map and fillna
transform_data['Type_fuel'] = transform_data['Type_fuel'].map(fuel_type_map)

# Temporal features engineering

In [9]:
transform_data["policy_age"] = (snapshot_date - transform_data["Date_start_contract"]).dt.days
transform_data['driving_experience'] = (snapshot_date - transform_data['Date_driving_licence']).dt.days
transform_data['customer_age'] = ((snapshot_date - transform_data['Date_birth']).dt.days)/365.25
transform_data['vehicle_age'] = snapshot_date.year - transform_data['Year_matriculation']

# Claim features engineering

In [10]:
transform_data['claims_per_year'] = transform_data['N_claims_history'] / (transform_data['policy_age'] / 365.25 + 1e-6)
transform_data['recent_claims_rate'] = transform_data['N_claims_history'] / (transform_data['policy_age'] / 365.25 + 1e-6)
transform_data['avg_claim_cost'] = transform_data['Cost_claims_year'] / (transform_data['N_claims_year'] + 1e-6)
transform_data['log_claim_cost'] = np.log1p(transform_data['Cost_claims_year'])
transform_data['claim_frequency_increase'] = (transform_data['recent_claims_rate'] > transform_data['claims_per_year']).astype(int)

# Vehicle features engineering

In [11]:
transform_data['power_weight_ratio'] = transform_data['Power'] / transform_data['Weight']
transform_data['premum_to_value'] = transform_data['Premium'] / transform_data['Value_vehicle']
transform_data['value_per_kg'] = transform_data['Value_vehicle'] / transform_data['Weight']
transform_data['vehicle_age'] = transform_data['Year_matriculation'].apply(lambda x: snapshot_date.year - x)
transform_data['urban_high_risk'] = ((transform_data['Area'] == 1) & (transform_data['Type_risk'] == 1)).astype(int)
transform_data['value_per_power'] = transform_data['Value_vehicle'] / (transform_data['Power'] + 1e-6)

# Fill the empty cells with the appropriate values

## Fuel type
use simple prediction

In [12]:
transform_data.loc[(transform_data['Type_fuel'].isna()) & (transform_data['Type_risk'] == 1), 'Type_fuel'] = 0
transform_data.loc[(transform_data['Type_fuel'].isna()) & (transform_data['Type_risk'] == 4), 'Type_fuel'] = 1
if transform_data['Type_fuel'].isna().any():
    transform_data['Type_fuel'].fillna(0, inplace=True)

## Length

In [13]:
features = ['power_weight_ratio', 'Power', 'Cylinder_capacity', 'Weight', 'Type_risk', 'N_doors', 'Type_fuel',]
X = transform_data[~transform_data['Length'].isna()][features]
y = transform_data[~transform_data['Length'].isna()]['Length']

model = ElasticNetCV(cv=5, random_state=42, n_jobs=-1)
model.fit(X, y)

if model.score(X, y) < 0.5:
    raise ValueError("Imputation model performance unacceptable")
else:
    missing_mask = transform_data['Length'].isna()
    transform_data.loc[missing_mask, 'Length'] = model.predict(transform_data[missing_mask][features])

transform_data['Length'] = transform_data['Length'].round(3)

ElasticNetCV cause use L1 and L2 regularization

## Is lapse

In [14]:
transform_data['in_lapse'] = transform_data['Date_lapse'].notna().astype(int)

# Drop the unnecessary columns

In [15]:
columns_to_drop = ["Date_start_contract", "Date_last_renewal", "Date_birth", "Date_driving_licence", 'Date_next_renewal', 'Date_lapse']
transform_data.drop(columns=columns_to_drop, inplace=True)

# Check Null values

In [16]:
transform_data.isnull().sum()

Distribution_channel        0
Seniority                   0
Policies_in_force           0
Max_policies                0
Max_products                0
Lapse                       0
Payment                     0
Premium                     0
Cost_claims_year            0
N_claims_year               0
N_claims_history            0
R_Claims_history            0
Type_risk                   0
Area                        0
Second_driver               0
Year_matriculation          0
Power                       0
Cylinder_capacity           0
Value_vehicle               0
N_doors                     0
Type_fuel                   0
Length                      0
Weight                      0
policy_age                  0
driving_experience          0
customer_age                0
vehicle_age                 0
claims_per_year             0
recent_claims_rate          0
avg_claim_cost              0
log_claim_cost              0
claim_frequency_increase    0
power_weight_ratio          0
premum_to_

# Flag High-Correlation Features
remove the weak features

In [17]:
corr_matrix = transform_data.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.85)]
print(f"Kill these redundant features: {to_drop}")
transform_data.drop(columns=to_drop, inplace=True)

transform_data.isnull().sum()

Kill these redundant features: ['Weight', 'customer_age', 'vehicle_age', 'recent_claims_rate', 'avg_claim_cost']


Distribution_channel        0
Seniority                   0
Policies_in_force           0
Max_policies                0
Max_products                0
Lapse                       0
Payment                     0
Premium                     0
Cost_claims_year            0
N_claims_year               0
N_claims_history            0
R_Claims_history            0
Type_risk                   0
Area                        0
Second_driver               0
Year_matriculation          0
Power                       0
Cylinder_capacity           0
Value_vehicle               0
N_doors                     0
Type_fuel                   0
Length                      0
policy_age                  0
driving_experience          0
claims_per_year             0
log_claim_cost              0
claim_frequency_increase    0
power_weight_ratio          0
premum_to_value             0
value_per_kg                0
urban_high_risk             0
value_per_power             0
in_lapse                    0
dtype: int

try to make spline-transformed of vehicle age

In [18]:
transform_data['vehicle_age'] = (snapshot_date.year - transform_data['Year_matriculation'].values.reshape(-1, 1))
spline = SplineTransformer(n_knots=5, degree=2, include_bias=False)
transform_data['vehicle_age'] = spline.fit_transform(transform_data['vehicle_age'].values.reshape(-1, 1))

In [19]:
numeric_data = transform_data.select_dtypes(include=['number'])

corr_matrix = numeric_data.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.85)]
print(f"Kill these redundant features: {to_drop}")
transform_data.drop(columns=to_drop, inplace=True)

transform_data.isnull().sum()

Kill these redundant features: []


Distribution_channel        0
Seniority                   0
Policies_in_force           0
Max_policies                0
Max_products                0
Lapse                       0
Payment                     0
Premium                     0
Cost_claims_year            0
N_claims_year               0
N_claims_history            0
R_Claims_history            0
Type_risk                   0
Area                        0
Second_driver               0
Year_matriculation          0
Power                       0
Cylinder_capacity           0
Value_vehicle               0
N_doors                     0
Type_fuel                   0
Length                      0
policy_age                  0
driving_experience          0
claims_per_year             0
log_claim_cost              0
claim_frequency_increase    0
power_weight_ratio          0
premum_to_value             0
value_per_kg                0
urban_high_risk             0
value_per_power             0
in_lapse                    0
vehicle_ag

# Feature scaling
remove outliers

In [20]:
scaler = RobustScaler()

numeric_features = ['Power', 'Cylinder_capacity', 'Value_vehicle', 'Premium', 'N_claims_year', 'N_claims_history', 'R_Claims_history', 'Cost_claims_year', 'power_weight_ratio', 'value_per_kg', 'value_per_power', 'policy_age', 'driving_experience']
transform_data[numeric_features] = scaler.fit_transform(transform_data[numeric_features])

# Target Engineering

In [21]:
transform_data['risk_score'] = (
    0.6 * (transform_data['Cost_claims_year'] / transform_data['Premium']) + 
    0.4 * transform_data['claims_per_year']
)
print(transform_data['risk_score'].describe())

count    1.055470e+05
mean    -1.058874e+02
std      6.513737e+04
min     -1.871224e+07
25%      0.000000e+00
50%      9.238064e-02
75%      4.274453e-01
max      3.363061e+06
Name: risk_score, dtype: float64


# Visualization distribution

In [22]:
def visualize_data(data):
    """
    Visualizes the data by plotting the distribution of each numerical column.

    Parameters:
    data (DataFrame): The DataFrame containing the data to visualize.
    """
    # Set up the figure size for better visibility
    plt.figure(figsize=(20, 15))

    # Get numerical columns (excluding date columns)
    numerical_cols = data.select_dtypes(include=['int64', 'float64']).columns

    # Create subplots for each numerical column
    for i, col in enumerate(numerical_cols, 1):
        plt.subplot(7, 7, i)
        sns.histplot(data=data, x=col, kde=True)
        plt.title(f'Distribution of {col}')
        plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [23]:
# visualize_data(raw_data)

In [24]:
# visualize_data(transform_data)